In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.chdir("C:\\Users\\MANDY\\Desktop\\fina")

In [8]:
df2=pd.read_csv('application_train.csv')

In [3]:
# df2.shape

In [4]:
# df2.AMT_REQ_CREDIT_BUREAU_QRT[(df2.AMT_REQ_CREDIT_BUREAU_QRT.isnull())]= 0
# df2.AMT_REQ_CREDIT_BUREAU_HOUR[(df2.AMT_REQ_CREDIT_BUREAU_HOUR.isnull())]= 0
# df2.AMT_REQ_CREDIT_BUREAU_YEAR[(df2.AMT_REQ_CREDIT_BUREAU_YEAR.isnull())]= 0
# df2.AMT_REQ_CREDIT_BUREAU_DAY[(df2.AMT_REQ_CREDIT_BUREAU_DAY.isnull())]= 0
# df2.AMT_REQ_CREDIT_BUREAU_MON[(df2.AMT_REQ_CREDIT_BUREAU_MON.isnull())]= 0
# df2.AMT_REQ_CREDIT_BUREAU_WEEK[(df2.AMT_REQ_CREDIT_BUREAU_WEEK.isnull())]= 0

In [5]:
# df2.to_csv("train_imputed_done.csv")

In [11]:
bureau_balance = pd.read_csv('bureau_balance.csv')
# pos_cash_balance = pd.read_csv('POS_CASH_balance_imputed.csv')
# bureau = pd.read_csv('bureau_imputed.csv')
previous_app = pd.read_csv('IMPUTED_previous_application.csv')
app_train = pd.read_csv('train_imputed_done.csv')
app_test = pd.read_csv('application_test.csv')
credit_card_balance = pd.read_csv('credit_card_balance_imputed.csv')
installments_payments = pd.read_csv('installments_payments_imputed.csv')

FileNotFoundError: File b'IMPUTED_previous_application.csv' does not exist

In [ ]:
app_train.head()

In [ ]:
app_train.drop("Unnamed: 0",inplace=True,axis=1)
app_train.drop("Unnamed: 0.1",inplace=True,axis=1)
#app_train.drop(" ",inplace=True,axis=1)

In [ ]:
app_train.head()

In [ ]:
# bureau_balance = bureau_balance.loc[:,(bureau_balance.isnull().mean() < 0.2)]
# pos_cash_balance = pos_cash_balance.loc[:,(pos_cash_balance.isnull().mean() < 0.2)]

# bureau = bureau.loc[:,(bureau.isnull().mean() < 0.2)]
# previous_app = previous_app.loc[:,(previous_app.isnull().mean() < 0.2)]
# app_train = app_train.loc[:,(app_train.isnull().mean() < 0.2)]
# #app_test = pd.read_csv('application_test.csv')
# credit_card_balance = credit_card_balance.loc[:,(credit_card_balance.isnull().mean() < 0.2)]
# installments_payments = installments_payments.loc[:,(installments_payments.isnull().mean() < 0.2)]

In [ ]:
app_train.shape

In [ ]:
# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

In [ ]:
def normalize_categorical(df, group_var, col_name):
    
    """Computes counts and normalized counts for each observation
    of `group_var` for each unique category in every categorical variable
    
    Parameters 
    ----------
    df - DataFrame for which we will calculate count
    
    group_var  = string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    col_name = string
            Variable added to the front of column names to keep track of columns
            
            """
    # select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))
    
    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]
    
    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])                                              
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (col_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical
    

In [ ]:
bureau_counts = normalize_categorical(bureau, group_var = 'SK_ID_CURR', col_name = 'bureau')
bureau_counts.head()

In [ ]:
# Grouping data  so  that we can merge all the files in 1 dataset

data_bureau_agg=bureau.groupby(by='SK_ID_CURR').mean()
data_credit_card_balance_agg=credit_card_balance.groupby(by='SK_ID_CURR').mean()
data_previous_application_agg=previous_app.groupby(by='SK_ID_CURR').mean()
data_installments_payments_agg=installments_payments.groupby(by='SK_ID_CURR').mean()
data_POS_CASH_balance_agg=pos_cash_balance.groupby(by='SK_ID_CURR').mean()

data_bureau_agg.head()

In [ ]:
#train = app_train.join(data_bureau_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
#train = app_train.join(bureau_counts, on = 'SK_ID_CURR', how = 'left')
# df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
# df = df.join(data_credit_card_balance_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
# df = df.join(data_previous_application_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
# df = df.join(data_installments_payments_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')

count_nan = (len(train) - train.count())*100/len(train)
count_nan.to_excel("null3.xlsx")

In [ ]:
data_bureau_agg.shape

In [ ]:
bureau_counts.head()
#data_bureau_agg.drop("Unnamed: 0",inplace=True,axis=1)

In [ ]:
count_nan = (len(data_bureau_agg) - data_bureau_agg.count())*100/len(data_bureau_agg)
count_nan.to_excel("null4.xlsx")

In [ ]:
def merge(df):
    df = df.join(data_bureau_agg, how='inner', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    df = df.join(bureau_counts, on = 'SK_ID_CURR', how = 'inner')
    df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'inner')
    df = df.join(data_credit_card_balance_agg, how='inner', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
    df = df.join(data_previous_application_agg, how='inner', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
    df = df.join(data_installments_payments_agg, how='inner', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    
    return df

train = merge(app_train)
#test = merge(app_test)
display(train.head())

In [ ]:
train.shape

In [ ]:
train.to_csv("merge_done_with_null.csv")

In [ ]:
count_nan = (len(train) - train.count())*100/len(train)
count_nan.to_excel("null3.xlsx")

In [ ]:
# Create a label encoder object
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0


# Iterate through the columns
for col in train:
    if train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train[col].unique())) <= 2:
            # Train on the training data
            le.fit(train[col])
            # Transform both training and testing data
            train[col] = le.transform(train[col])
            #test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1

print('%d columns were label encoded.' % le_count)

In [ ]:
all_data = pd.get_dummies(train)

display(all_data.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(all_data.drop(columns='TARGET'), all_data['TARGET'], test_size = 0.2, random_state = 200)
print("X Training shape", X_train.shape)
print("X Testing shape", X_test.shape)
print("Y Training shape", Y_train.shape)
print("Y Testing shape", Y_test.shape)

In [ ]:


logreg = LogisticRegression(random_state=0, class_weight='balanced', C=100)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict_proba(X_test)[:,1]

#Y_pred_proba = logreg.predict_proba(X_test)

print('Train/Test split results:')
#print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(Y_test, Y_pred))
print("ROC",  roc_auc_score(Y_test, Y_pred))